# NLP

## Table of Contents
1. [Class Distribution](#class)
2. [Clean and Tokenize](#clean)

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from nltk.corpus import stopwords
import string

In [ ]:
df = pd.read_pickle('reviews.pkl')

In [ ]:
df.head()

## Class Distribution

How many reviews do we have in each category?

In [ ]:
df['Score'].value_counts()

In [ ]:
sns.countplot(df['Score'], alpha =.80, palette= ['green','red'])
plt.title('Positive vs Negative Reviews', fontweight='bold')
plt.ylabel('Count')
plt.show()

## Clean and Tokenize <a id='clean'></a>

Let's see what our text looks like so we can clean and tokenize it properly.

In [ ]:
print(df['Review'][0])

To clean the strings, we'll remove punctuation and capitalization, so that all words will be counted correctly.

In [ ]:
stopwords_list = stopwords.words('english')+list(string.punctuation)
print(stopwords_list)

In [ ]:
# print(df['Review'][0])

Next, we'll tokenize the reviews.

In [ ]:
# df['Review'] = df['Review'].apply(lambda x: word_tokenize(x))

In [ ]:
# print(df['Review'][0])